In [69]:
text="""A banana is an elongated, edible fruit  botanically a berry[1]  produced by several kinds of large herbaceous flowering plants in the genus Musa. In some countries, cooking bananas are called plantains, distinguishing them from dessert bananas. The fruit is variable in size, color and firmness, but is usually elongated and curved, with soft flesh rich in starch covered with a peel, which may have a variety of colors when ripe. It grows upward in clusters near the top of the plant. Almost all modern edible seedless (parthenocarp) cultivated bananas come from two wild species – Musa acuminata and Musa balbisiana, or hybrids of them. Musa species are native to tropical Indomalaya and Australia; they were probably domesticated in New Guinea. They are grown in 135 countries, primarily for their fruit, and to a lesser extent to make banana paper and textiles, while some are grown as ornamental plants. The world's largest producers of bananas in 2022 were India and China, which together accounted for approximately 26% of total production. Bananas are eaten raw or cooked in recipes varying from curries to banana chips, fritters, fruit preserves, or simply baked or steamed. Worldwide, there is no sharp distinction between dessert "bananas" and cooking "plantains": this works well enough in the Americas and Europe, but it breaks down in Southeast Asia where many more kinds of bananas are grown and eaten. The term "banana" is applied also to other members of the genus Musa, such as the scarlet banana (Musa coccinea), the pink banana (Musa velutina), and the Fe'i bananas. Members of the genus Ensete, such as the snow banana (Ensete glaucum) and the economically important false banana (Ensete ventricosum) of Africa are sometimes included. Both genera are in the banana family, Musaceae. Banana plantations are subject to damage by parasitic nematodes and insect pests, and to fungal and bacterial diseases, one of the most serious being Panama disease which is caused by a Fusarium fungus. This and black sigatoka threaten the production of Cavendish bananas, the main kind eaten in the Western world, which is a triploid Musa acuminata. Plant breeders are seeking new varieties, but these are difficult to breed given that commercial varieties are seedless. To enable future breeding, banana germplasm is conserved in multiple gene banks around the world."""

In [70]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

r_splitter = RecursiveCharacterTextSplitter(
    separators = ["\n\n", "\n", " "],  # List of separators based on requirement (defaults to ["\n\n", "\n", " "])
    chunk_size = 100,  # size of each chunk created
    chunk_overlap  = 0,  # size of  overlap between chunks in order to maintain the context
    length_function = len  # Function to calculate size, currently we are using "len" which denotes length of string however you can pass any token counter)
)

In [71]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

In [72]:
chunks = r_splitter.split_text(text)

for chunk in chunks:
    print(len(chunk))

98
92
93
98
93
93
92
99
99
96
96
98
95
93
97
93
98
95
98
99
95
98
99
92
50


In [73]:
import os
import pickle
import time
import langchain
from langchain_openai import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from dotenv import load_dotenv

In [74]:
load_dotenv()

True

In [75]:
llm=OpenAI(temprature=0.9,max_tokens=500)

/home/zigron/anaconda3/envs/codeenv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3517: UserWarning: WARNING! temprature is not default parameter.
                temprature was transferred to model_kwargs.
                Please confirm that temprature is what you intended.
  if await self.run_code(code, result, async_=asy):


In [76]:
urls = [
    "https://en.wikipedia.org/wiki/Banana",
    "https://en.wikipedia.org/wiki/Apple",
]

In [77]:
loaders=UnstructuredURLLoader(
    urls=urls
)
data = loaders.load()

In [78]:
len(data)

2

In [79]:
r_splitter = RecursiveCharacterTextSplitter(
    separators = ["\n\n", "\n", " "],
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len
)

In [80]:
chunks=r_splitter.split_documents(data)

In [81]:
len(chunks)

1865

In [82]:
chunks[18]

Document(metadata={'source': 'https://en.wikipedia.org/wiki/Banana'}, page_content='Taqbaylit\n\nTayal\n\nతెలుగు\n\nไทย\n\nТоҷикӣ\n\nTsetsêhestâhese\n\nತುಳು\n\nTürkçe\n\nTyap\n\nУкраїнська\n\nاردو')

In [83]:
embeddings=OpenAIEmbeddings()
vectors=FAISS.from_documents(chunks,embeddings)

In [85]:
#chain=RetrievalQAWithSourcesChain(llm=llm,retriever=vectors.as_retriever())

In [86]:
#query = "what is the price of Tiago iCNG?"

#langchain.debug=True

#chain({"question": query}, return_only_outputs=True)